In [ ]:
import pandas as pd
import re, subprocess, pyodbc, os
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  

In [ ]:
df = pd.read_csv(r'c:\Users\Tim\Documents\CIO\AppCsec6.csv') 

In [ ]:
df['IDTEXT'] = df.TEXT.apply(lambda s : re.search('\d{1,2}\.\d{1,2}(\.\d{1,2})?', s).group() );
df['QTEXT'] = df.TEXT.apply(lambda s : re.sub('^\d{1,2}\.\d{1,2}(\.\d{1,2})?', '', s) );
df[['IDTEXT', 'QTEXT']].to_csv(r'c:\Users\Tim\Documents\CIO\AppCsec6_text.csv', index=False)
df[['IDTEXT', 'QTEXT']].head(0)

engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT CONVERT(NVARCHAR(5) , PK_QuestionTypeId) as QTID, code QCODE  FROM fsma_QuestionTypes
""" 
qtypes = pd.read_sql(query,con=conn)  
qtypes.head(2)

In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """\
SELECT PK_PickList, PK_PickListType, DisplayValue, CodeValue, SortPos, Description, UsageField FROM vwPickLists
WHERE PK_PickListType IN (100,101,102,103,104,105,106,113,114,164,165,166)
""" 
 
dfpicks = pd.read_sql(query,con=conn) 
UsageField=dfpicks.iloc[0]['UsageField']
s=f"\n"
s = s + f"Enum PICK_{UsageField} \t'{dfpicks.iloc[0]['Description']}-{dfpicks.iloc[0]['PK_PickListType']}\n"
for i,r in dfpicks.iterrows():
    if UsageField != dfpicks.iloc[i]['UsageField']:
        UsageField=dfpicks.iloc[i]['UsageField']
        s = s + f"End Enum\n"
        s = s + f"Enum PICK_{UsageField} \t'{dfpicks.iloc[i]['Description']}-{dfpicks.iloc[i]['PK_PickListType']}\n"
    s = s + f"\t{r.CodeValue} = {str(r.PK_PickList)} \t'{str(r.DisplayValue)[:25]}\n" 
s=s+"End Enum\n"
print(s)
conn.close()

In [ ]:
engine = create_engine(connstr)
conn = engine.connect()
query = """\
SELECT PK_PickList, PK_PickListType, DisplayValue, CodeValue, SortPos, Description, UsageField FROM vwPickLists
WHERE PK_PickListType IN (100,101,102,103,104,105,106,113,114,164,165,166)
""" 
 
dfpicks = pd.read_sql(query,con=conn) 
UsageField=dfpicks.iloc[0]['UsageField']
s=f"\n"  
for i,r in dfpicks.iterrows():
    UsageField = r.UsageField
    s = s + f"\tDECLARE @PICK_{UsageField}_{r.CodeValue} INT = {str(r.PK_PickList)}\n" 
print(s)
conn.close()

In [56]:
plt = re.findall('PK_PickListType="(\d{1,4})"','PK_PickListType="114" PK_PickListType="112"')
plt

['114', '112']

,path,fn,ext,plt,content
0,C:\_som\_src\_compile\saop\src\2020_A_SAOP_1.aspx,2020_A_SAOP_1,aspx,"[100, 101, 102]","<%@ Page Title="""" Language=""vb"" AutoEventWireu..."
1,C:\_som\_src\_compile\saop\src\2020_A_SAOP_1.a...,2020_A_SAOP_1,aspx.designer.vb,[],'---------------------------------------------...
2,C:\_som\_src\_compile\saop\src\2020_A_SAOP_1.a...,2020_A_SAOP_1,aspx.vb,[],Imports CyberBalance.VB.Core\nImports CyberBal...
3,C:\_som\_src\_compile\saop\src\2020_A_SAOP_10....,2020_A_SAOP_10,aspx,[],"<%@ Page Title="""" Language=""vb"" AutoEventWireu..."
4,C:\_som\_src\_compile\saop\src\2020_A_SAOP_10....,2020_A_SAOP_10,aspx.designer.vb,[],'---------------------------------------------...
5,C:\_som\_src\_compile\saop\src\2020_A_SAOP_10....,2020_A_SAOP_10,aspx.vb,[],Imports CyberBalance.VB.Core\nImports CyberBal...
6,C:\_som\_src\_compile\saop\src\2020_A_SAOP_11....,2020_A_SAOP_11,aspx,[],"<%@ Page Title="""" Language=""vb"" AutoEventWireu..."
7,C:\_som\_src\_compile\saop\src\2020_A_SAOP_11....,2020_A_SAOP_11,aspx.designer.vb,[],'---------------------------------------------...
8,C:\_som\_src\_compile\saop\src\2020_A_SAOP_11....,2020_A_SAOP_11,aspx.vb,[],Imports CyberBalance.VB.Core\nImports CyberBal...
9,C:\_som\_src\_compile\saop\src\2020_A_SAOP_12....,2020_A_SAOP_12,aspx,[113],"<%@ Page Title="""" Language=""vb"" AutoEventWireu..."
